In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('B2W-Reviews01.csv',sep = ';')
df.set_index(['submission_date'],inplace = True)
df.info()

/home/saraiva/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Index: 132373 entries, 2018-01-01 00:11:28 to 2018-05-31 23:50:33
Data columns (total 13 columns):
reviewer_id              132373 non-null object
product_id               132373 non-null object
product_name             132289 non-null object
product_brand            40982 non-null object
site_category_lv1        132367 non-null object
site_category_lv2        128360 non-null object
review_title             132373 non-null object
overall_rating           132373 non-null int64
recommend_to_a_friend    132355 non-null object
review_text              132373 non-null object
reviewer_birth_year      126389 non-null float64
reviewer_gender          128237 non-null object
reviewer_state           128382 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 14.1+ MB


In [3]:
df = df[['review_text','reviewer_gender','reviewer_birth_year',
         'reviewer_state','overall_rating','recommend_to_a_friend']]

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import unidecode
sw = stopwords.words('portuguese')

In [ ]:
text_data = df['review_text']
text_data = [phrases.lower() for phrases in text_data]
text_data = [unidecode.unidecode(phrases) for phrases in text_data]

rgx_tokenizer = RegexpTokenizer('\w+')
text_data = [rgx_tokenizer.tokenize(phrases) for phrases in text_data]

sentences = list()
corpus = list()

for phrases in text_data:
    sentences = list()
    for word in phrases:
        if word not in sw:
            sentences.append(word)
    corpus.append(sentences)

text_data = corpus.copy()
del(corpus)
df['text_data'] = [' '.join(phrases) for phrases in text_data]

In [167]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [168]:
df = df.dropna()

In [169]:
X = df.drop('overall_rating',axis=1).reset_index(drop = True)
y = df['overall_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [246]:
tfidf = TfidfVectorizer()
model = RandomForestClassifier(n_estimators=15)

#List of (name, transformer, column(s))
ct = ColumnTransformer([('tfidf',tfidf,'text_data')],remainder='passthrough',sparse_threshold=1)

pipe = Pipeline(steps = [('ct',ct),('rf',rf)])

In [247]:
# tfidf.fit_transform(X_train['text_data'][:1000])
# ct.fit_transform(X_train[['text_data','reviewer_birth_year']][:1000]).toarray()

In [252]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'ct', 'rf', 'ct__n_jobs', 'ct__remainder', 'ct__sparse_threshold', 'ct__transformer_weights', 'ct__transformers', 'ct__verbose', 'ct__tfidf', 'ct__tfidf__analyzer', 'ct__tfidf__binary', 'ct__tfidf__decode_error', 'ct__tfidf__dtype', 'ct__tfidf__encoding', 'ct__tfidf__input', 'ct__tfidf__lowercase', 'ct__tfidf__max_df', 'ct__tfidf__max_features', 'ct__tfidf__min_df', 'ct__tfidf__ngram_range', 'ct__tfidf__norm', 'ct__tfidf__preprocessor', 'ct__tfidf__smooth_idf', 'ct__tfidf__stop_words', 'ct__tfidf__strip_accents', 'ct__tfidf__sublinear_tf', 'ct__tfidf__token_pattern', 'ct__tfidf__tokenizer', 'ct__tfidf__use_idf', 'ct__tfidf__vocabulary', 'rf__bootstrap', 'rf__class_weight', 'rf__criterion', 'rf__max_depth', 'rf__max_features', 'rf__max_leaf_nodes', 'rf__min_impurity_decrease', 'rf__min_impurity_split', 'rf__min_samples_leaf', 'rf__min_samples_split', 'rf__min_weight_fraction_leaf', 'rf__n_estimators', 'rf__n_jobs', 'rf__oob_score', 'rf__random_st

In [253]:
param_grid = {'ct__tfidf__ngram_range' : [(1,1),(1,2),(1,3)],
              'ct__tfidf__max_features' : [500],
              'rf__min_samples_split' :(3,15),
              'rf__max_depth' : (2,50)}

In [254]:
rs = RandomizedSearchCV(pipe,param_distributions=param_grid,cv = 3,verbose = 2)

In [255]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100984 entries, 49140 to 121958
Data columns (total 6 columns):
review_text              100984 non-null object
reviewer_gender          100984 non-null object
reviewer_birth_year      100984 non-null float64
reviewer_state           100984 non-null object
recommend_to_a_friend    100984 non-null object
text_data                100984 non-null object
dtypes: float64(1), object(5)
memory usage: 5.4+ MB


In [256]:
print(ct.get_params().keys())

dict_keys(['n_jobs', 'remainder', 'sparse_threshold', 'transformer_weights', 'transformers', 'verbose', 'tfidf', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary'])


In [257]:
rs.fit(X_train[['text_data','reviewer_birth_year']],y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] rf__min_samples_split=15, rf__max_depth=2, ct__tfidf__ngram_range=(1, 3), ct__tfidf__max_features=500 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  rf__min_samples_split=15, rf__max_depth=2, ct__tfidf__ngram_range=(1, 3), ct__tfidf__max_features=500, total=  22.4s
[CV] rf__min_samples_split=15, rf__max_depth=2, ct__tfidf__ngram_range=(1, 3), ct__tfidf__max_features=500 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.4s remaining:    0.0s


[CV]  rf__min_samples_split=15, rf__max_depth=2, ct__tfidf__ngram_range=(1, 3), ct__tfidf__max_features=500, total=   7.7s
[CV] rf__min_samples_split=15, rf__max_depth=2, ct__tfidf__ngram_range=(1, 3), ct__tfidf__max_features=500 
[CV]  rf__min_samples_split=15, rf__max_depth=2, ct__tfidf__ngram_range=(1, 3), ct__tfidf__max_features=500, total=   7.6s
[CV] rf__min_samples_split=3, rf__max_depth=50, ct__tfidf__ngram_range=(1, 2), ct__tfidf__max_features=500 
[CV]  rf__min_samples_split=3, rf__max_depth=50, ct__tfidf__ngram_range=(1, 2), ct__tfidf__max_features=500, total=  16.7s
[CV] rf__min_samples_split=3, rf__max_depth=50, ct__tfidf__ngram_range=(1, 2), ct__tfidf__max_features=500 
[CV]  rf__min_samples_split=3, rf__max_depth=50, ct__tfidf__ngram_range=(1, 2), ct__tfidf__max_features=500, total=  17.2s
[CV] rf__min_samples_split=3, rf__max_depth=50, ct__tfidf__ngram_range=(1, 2), ct__tfidf__max_features=500 
[CV]  rf__min_samples_split=3, rf__max_depth=50, ct__tfidf__ngram_range=(1, 

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  6.0min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ct',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='passthrough',
                                                                sparse_threshold=1,
                                                                transformer_weights=None,
                                                                transformers=[('tfidf',
                                                                               TfidfVectorizer(analyzer='word',
                                                                                               binary=False,
                                                                                               decode_error='strict',
                                                                        

In [258]:
pd.DataFrame(rs.cv_results_).sort_values(by = 'mean_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__min_samples_split,param_rf__max_depth,param_ct__tfidf__ngram_range,param_ct__tfidf__max_features,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
6,1.186603,0.005577,0.475641,0.010751,15,2,"(1, 1)",500,"{'rf__min_samples_split': 15, 'rf__max_depth':...",0.409411,0.394195,0.396405,0.400004,0.006713,10
7,3.363362,0.017594,0.827495,0.005373,3,2,"(1, 2)",500,"{'rf__min_samples_split': 3, 'rf__max_depth': ...",0.408163,0.417130,0.418301,0.414531,0.004528,9
3,6.371049,0.040847,1.157998,0.002014,3,2,"(1, 3)",500,"{'rf__min_samples_split': 3, 'rf__max_depth': ...",0.420075,0.412406,0.427837,0.420106,0.006300,8
0,11.330721,6.924956,1.244126,0.092280,15,2,"(1, 3)",500,"{'rf__min_samples_split': 15, 'rf__max_depth':...",0.403886,0.420546,0.452406,0.425612,0.020130,7
2,12.996484,0.160487,0.577274,0.003987,3,50,"(1, 1)",500,"{'rf__min_samples_split': 3, 'rf__max_depth': ...",0.551436,0.552182,0.551367,0.551662,0.000369,6
1,16.114821,0.254165,0.912808,0.007604,3,50,"(1, 2)",500,"{'rf__min_samples_split': 3, 'rf__max_depth': ...",0.556278,0.552657,0.551159,0.553365,0.002149,5
8,10.696730,0.204597,0.568684,0.004999,15,50,"(1, 1)",500,"{'rf__min_samples_split': 15, 'rf__max_depth':...",0.555120,0.553430,0.551961,0.553504,0.001291,4
5,18.887976,0.187502,1.250048,0.010181,3,50,"(1, 3)",500,"{'rf__min_samples_split': 3, 'rf__max_depth': ...",0.553783,0.555539,0.552377,0.553900,0.001294,3
4,13.171802,0.085142,0.923218,0.013086,15,50,"(1, 2)",500,"{'rf__min_samples_split': 15, 'rf__max_depth':...",0.554169,0.556311,0.552080,0.554187,0.001728,2
9,16.472694,0.366644,1.241799,0.003519,15,50,"(1, 3)",500,"{'rf__min_samples_split': 15, 'rf__max_depth':...",0.556219,0.554678,0.553981,0.554959,0.000935,1


### Criando meu próprio transformer :)

After you decided which one suits to your needs you subclass <b>BaseEstimator</b> and an appropriate class for your type (one of <b>ClassifierMixin, RegressorMixin, ClusterMixin, TransformerMixin</b>).

In [272]:
from sklearn.base import BaseEstimator, TransformerMixin

class QuickTextPreProcessing(BaseEstimator, TransformerMixin):  
    """An example of classifier"""

    def __init__(self):
        """
        Called when initializing the classifier
        """


    def fit(self, X, y=None):

        text_data = X
        text_data = [phrases.lower() for phrases in text_data]
        text_data = [unidecode.unidecode(phrases) for phrases in text_data]

        rgx_tokenizer = RegexpTokenizer('\w+')
        text_data = [rgx_tokenizer.tokenize(phrases) for phrases in text_data]

        sentences = list()
        corpus = list()

        for phrases in text_data:
            sentences = list()
            for word in phrases:
                if word not in sw:
                    sentences.append(word)
            corpus.append(sentences)
            
        text_data = corpus

        return self

    def transform(self, X, y=None):
        try:
            return text_data
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        return([self._meaning(x) for x in X])

    def score(self, X, y=None):
        # counts number of values bigger than mean
        return(sum(self.predict(X))) 

In [273]:
qtpp = QuickTextPreProcessing()

In [277]:
qtpp.fit_transform(df['review_text'])

[['contente',
  'compra',
  'entrega',
  'rapida',
  'unico',
  'problema',
  'americanas',
  'troca',
  'devolucao',
  'produto',
  'consumidor',
  'problemas',
  'espera'],
 ['apenas',
  'r',
  '1994',
  '20',
  'consegui',
  'comprar',
  'lindo',
  'copo',
  'acrilico'],
 ['supera',
  'agilidade',
  'praticidade',
  'outras',
  'panelas',
  'eletricas',
  'costumo',
  'usar',
  'outra',
  'panela',
  'cozimento',
  'arroz',
  'japonesa',
  'leva',
  'tempo',
  '50',
  'minutos',
  'nessa',
  'panela',
  'rapido',
  'exatamente',
  '6',
  'minutos',
  'recomendo'],
 ['filho', 'amou', 'parece', 'verdade', 'tantos', 'detalhes'],
 ['entrega',
  'prazo',
  'americanas',
  'estao',
  'parabens',
  'smart',
  'tv',
  'boa',
  'navegacao',
  'internete',
  'aplicativos',
  'excelente',
  'nao',
  'trava',
  'falar',
  'imagem',
  'surpreender',
  'recomendo'],
 ['excelente',
  'produto',
  'material',
  'acrilico',
  'super',
  'resistente',
  'dentro',
  'adamantio',
  'faz',
  'milagre',


In [275]:
qtpp.fit(df['review_text'])

QuickTextPreProcessing()

In [ ]:
qtpp.transform(df['review_text'])